In [ ]:
from PosSelect_Functions_Old import *
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import pandas as pd
import numpy as np
import copy
import seaborn as sns
from scipy.stats import mannwhitneyu as mwu
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import fdrcorrection
from scipy.stats import wilcoxon
from scipy.optimize import curve_fit
from scipy.stats import fisher_exact
from scipy.stats import norm
import os
from collections import Counter
from scipy.stats import binomtest
from scipy.stats import spearmanr,pearsonr

hfont = {'fontname':'Arial'}
plt.rcParams["font.family"] = "Arial"

#Code borrowed heavily from here: https://stackoverflow.com/questions/62375034/find-non-overlapping-area-between-two-kde-plots
plt.rcParams.update(
    {"text.usetex": False}
)

def parse_table(i):
    return [int(x) for x in i.replace("[", "").replace("]", "").split(",")]

d_abrev = {"LiangSteinNeuron":"FC exc. neur.", "FetalChondrocytes":"F chond.", "SertoliMale":"FG sertoli", "preGC_IIaFemale":"FG preGC IIa",\
          "NeuralFemale":"FG neur.", "FetalGonadImmuneFemale":"FG immune", "VIP":"AC VIP inh. neur.", "LiangSteinProgenitor":"FC prog.",\
          "AdultHeartVentricularCardiomyocyte":"AH cardiomyo.", "AdultLoopOfHenle":"AK loop of henle", "FetalBrainNeurGlioblast_CB_VZ":"FCB glioblast",\
         "AdultProximalTubule":"AK prox. tub.", "FetalLeydigMale":"FG leydig", "SST":"AC SST inh neur.", "KosoyRoussosControlMicroglia":"AC microglia",\
         "FetalBrainFloorPlate":"FB fl. plate", "FetalArterialECs":"FH endoth.", "ASCT":"AC astro.", "FetalBrainCOP":"FB COP",\
         "AMY":"AA neur.", "PVALB":"AC PVALB inh neur.", "ITL23":"AC L2-3 IT neur.", "FetalBrainNeurCB_GNP_IPC_1":"FB inter. prog.", "FetalBrainNeurDAergic":"FB DA neur.",\
          "OGC":"AC Oligo.", "D1Pu":"AP D1 inh neur.", "FetalBrainNeurSerotonergic":"FB 5-HT neur.", "FetalBrainNeurDRG_2":"FS DRG neur.",\
          "FetalHeartPericytes":"FH peri.", "FetalHeartEndocardium":"FH endocard.", "FetalHeartCardiacFibroblasts":"FH fibro.", "FetalBrainNeurPurkinje_6":"FCB Purk. inh neur.",\
          "AdultHeartSmoothMuscle":"AH smooth musc.", "FetalBrainRoofPlate":"FB ro. plate"}

In [ ]:
#out = []
for fold in os.listdir("PerGene_Windows_TestBGS"):
    if "." not in fold:
        cell_type = fold
        #Read in the data
        dfw = pd.read_csv("PerGene_Windows_TestBGS/" + fold + "/PGW_BGS_PerWindow_NonCod_Dl" + fold + "_abs_logfc_All_MAFCut0.25_WS25000_SpecSup0.csv.gz")
        df = pd.read_csv("PerGene_Windows_TestBGS/" + cell_type + "/PGW_BGS_PerGene_NonCod_Dl" + cell_type + "_abs_logfc_All_MAFCut0.25_WS25000_SpecSup0.csv.gz")
        
        #Filter
        df = df[(df["Number Fixed Variants"] > 100) & (df["Number Polymorphic Variants"] > 50)]

        #Compute statistic
        df["Stat"] = -np.log10(df["Fisher exact p-value"])*np.sign(df["alpha"])
        dfw["Stat"] = -np.log10(dfw["Fisher exact p-value"])*np.sign(dfw["alpha"])

        #Restrict to this cutoff
        df2 = df[df["Proportion"] == 0.6]
        dfw2 = dfw[dfw["Proportion"] == 0.6]

        #For real and shuffled positions, compute per gene average across windows
        for r in ["Real", "ShuffledPositions"]:
            dfw3 = dfw2[dfw2["ShufOrReal"] == r]
            new = []

            for gene in np.unique(dfw3["Term"]):
                dfw3g = dfw3[dfw3["Term"] == gene]
                new.append([gene, np.mean(dfw3g["Stat"])])
            dff = pd.DataFrame(new).set_index(0)
            dff.columns = ["Window Mean"]
            dff = dff.join(df2.set_index("Term"))
            dff = dff.dropna()
            out.append([d_abrev[fold], r, pearsonr(dff["Window Mean"], dff["Stat"])[0]])
dfp = pd.DataFrame(out)

In [ ]:
#Plot correlations
dfp = pd.DataFrame(out)

fig, ax = plt.subplots(figsize = (18, 6))

dfp.columns = ["Cell type", "Shuffled positions", "Pearson's rho"]
dfp["Shuffled positions"] = dfp["Shuffled positions"].replace("Real", "Unshuffled")
dfp["Shuffled positions"] = dfp["Shuffled positions"].replace("ShuffledPositions", "Shuffled")
sns.barplot(data = dfp, x = "Cell type", y = "Pearson's rho", hue = "Shuffled positions")

plt.ylabel("Pearson's rho", size = 22)
plt.xlabel("Cell type", size = 22)
plt.xticks(size = 16)
plt.yticks(size = 16)

plt.ylim([0, 0.85])
plt.title("Testing for the effect of background selection across cell types", size = 26)
plt.xticks(rotation = 90)
plt.legend(fontsize = 15)

In [ ]:
#Used to plot correlation for an individual cell type
cell_type = "LiangSteinNeuron"

dfw = pd.read_csv("PerGene_Windows_TestBGS/" + cell_type + "/PGW_BGS_PerWindow_NonCod_Dl" + cell_type + "_abs_logfc_All_MAFCut0.25_WS25000_SpecSup0.csv.gz")
df = pd.read_csv("PerGene_Windows_TestBGS/" + cell_type + "/PGW_BGS_PerGene_NonCod_Dl" + cell_type + "_abs_logfc_All_MAFCut0.25_WS25000_SpecSup0.csv.gz")

df = df[(df["Number Fixed Variants"] > 100) & (df["Number Polymorphic Variants"] > 50)]

df["Stat"] = -np.log10(df["Fisher exact p-value"])*np.sign(df["alpha"])
dfw["Stat"] = -np.log10(dfw["Fisher exact p-value"])*np.sign(dfw["alpha"])

df2 = df[df["Proportion"] == 0.6]
dfw2 = dfw[dfw["Proportion"] == 0.6]
dfw2 = dfw2[dfw2["ShufOrReal"] == "Real"]
new = []

for gene in np.unique(dfw2["Term"]):
    dfw2g = dfw2[dfw2["Term"] == gene]
    new.append([gene, np.mean(dfw2g["Stat"])])
dff = pd.DataFrame(new).set_index(0)
dff.columns = ["Window Mean"]
dff = dff.join(df2.set_index("Term"))
dff = dff.dropna()

In [ ]:
dff

In [ ]:
print(pearsonr(dff["Window Mean"], dff["Stat"]))
print(spearmanr(dff["Window Mean"], dff["Stat"]))
sns.regplot(x = dff["Window Mean"], y = dff["Stat"])
plt.xlabel("Aggregated estimate", size = 16)
plt.ylabel("Per-window average estimate", size = 16)
plt.title("Unshuffled positions", size = 18)

In [ ]:
#This is for shuffling the positions
dff = dff.dropna()
print(pearsonr(dff["Window Mean"], dff["Stat"]))
print(spearmanr(dff["Window Mean"], dff["Stat"]))

sns.regplot(x = dff["Window Mean"], y = dff["Stat"])
plt.xlabel("Aggregated estimate", size = 16)
plt.ylabel("Per-window average estimate", size = 16)
plt.title("Shuffled positions", size = 18)

In [ ]:
#Repeating for PhyloP
dfw = pd.read_csv("PerGene_Windows_TestBGS/PGW_BGS_PerWindow_NonCod_PhyloP447_MAFCut0.25_WS25000_SpecSup250.csv.gz")
df = pd.read_csv("PerGene_Windows_TestBGS/PGW_BGS_PerGene_NonCod_PhyloP447_MAFCut0.25_WS25000_SpecSup250.csv.gz")

df = df[(df["Number Fixed Variants"] > 100) & (df["Number Polymorphic Variants"] > 50)]

df["Stat"] = -np.log10(df["Fisher exact p-value"])*np.sign(df["alpha"])
dfw["Stat"] = -np.log10(dfw["Fisher exact p-value"])*np.sign(dfw["alpha"])

df2 = df[df["Proportion"] == 0.6]
dfw2 = dfw[dfw["Proportion"] == 0.6]
dfw2 = dfw2[dfw2["ShufOrReal"] == "ShuffledPositions"]
new = []

for gene in np.unique(dfw2["Term"]):
    dfw2g = dfw2[dfw2["Term"] == gene]
    new.append([gene, np.mean(dfw2g["Stat"])])
dff = pd.DataFrame(new).set_index(0)
dff.columns = ["Window Mean"]
dff = dff.join(df2.set_index("Term"))
dff = dff.dropna()

In [ ]:
print(pearsonr(dff["Window Mean"], dff["Stat"]))
print(spearmanr(dff["Window Mean"], dff["Stat"]))
sns.regplot(x = dff["Window Mean"], y = dff["Stat"])

plt.xlabel("Aggregated estimate", size = 16)
plt.ylabel("Per-window average estimate", size = 16)
plt.title("Unshuffled positions", size = 18)


In [ ]:
#Testing out statistical power at various cutoffs
nsites = 1000

out = []

for i in [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]:
    bin1 = int(i*nsites)
    bin2 = int((1-i)*nsites)
    bin1_high = (1/1.23)*bin1
    bin2_high = bin2*1.23
    if i == 0.6:
        print([[bin1, bin1_high],[ bin2, bin2_high]])
    out.append([i, fisher_exact([[bin1, bin1_high],[ bin2, bin2_high]])[1]])

In [ ]:
fisher_exact([[bin1, bin1_high],[ bin2, bin2_high]])

In [ ]:
#Testing 
df = pd.DataFrame(out)
df[0] = 100*df[0]
df[1] = -np.log10(df[1])
df.columns = ["Percentile cutoff", "-log$_{10}$(p-value)"]

sns.scatterplot(data = df, x = "Percentile cutoff", y = "-log$_{10}$(p-value)")
plt.xlabel("Percentile cutoff", size = 16)
plt.ylabel("-log$_{10}$(p-value)", size = 16)
plt.title("Power holding odds ratio constant at 1.5", size = 18)